In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [2]:
!hdfs dfs -ls /tmp
!hdfs dfs -mkdir /tmp/wordcount
!hdfs dfs -ls /tmp/


Found 4 items
drwxrwx---   - root supergroup          0 2020-02-18 19:12 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-18 19:19 /tmp/hive
drwxrwxrwx   - root supergroup          0 2020-02-18 21:53 /tmp/output
drwxr-xr-x   - root supergroup          0 2020-02-18 19:38 /tmp/wordcount
mkdir: `/tmp/wordcount': File exists
Found 4 items
drwxrwx---   - root supergroup          0 2020-02-18 19:12 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-18 19:19 /tmp/hive
drwxrwxrwx   - root supergroup          0 2020-02-18 21:53 /tmp/output
drwxr-xr-x   - root supergroup          0 2020-02-18 19:38 /tmp/wordcount


In [3]:
!hdfs dfs -copyFromLocal data.tsv  /tmp/wordcount/

In [4]:
%%hive
DROP TABLE IF EXISTS t0;
CREATE TABLE t0 (
    c1 STRING,
    c2 ARRAY<CHAR(1)>, 
    c3 MAP<STRING, INT>
    )
    ROW FORMAT DELIMITED 
        FIELDS TERMINATED BY '\t'
        COLLECTION ITEMS TERMINATED BY ','
        MAP KEYS TERMINATED BY '#'
        LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE t0;

DROP TABLE IF EXISTS t0;
OK
Time taken: 27.255 seconds
CREATE TABLE t0 (
    c1 STRING,
    c2 ARRAY<CHAR(1)>, 
    c3 MAP<STRING, INT>
    )
    ROW FORMAT DELIMITED 
        FIELDS TERMINATED BY '\t'
        COLLECTION ITEMS TERMINATED BY ','
        MAP KEYS TERMINATED BY '#'
        LINES TERMINATED BY '\n';
OK
Time taken: 4.547 seconds
LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE t0;
Loading data to table default.t0
OK
Time taken: 4.03 seconds


In [7]:
%%hive
DROP TABLE IF EXISTS cont_ks;
CREATE TABLE cont_ks AS
SELECT a.k, COUNT(*)
FROM
(SELECT k
FROM t0
LATERAL VIEW explode(c3) tbl1 AS k, v) a
GROUP BY a.k;

DROP TABLE IF EXISTS cont_ks;
OK
Time taken: 0.023 seconds
CREATE TABLE cont_ks AS
SELECT a.k, COUNT(*)
FROM
(SELECT k
FROM t0
LATERAL VIEW explode(c3) tbl1 AS k, v) a
GROUP BY a.k;
Query ID = root_20200218225444_14cfb35c-552e-4958-93a6-74c9e6bb8cb7
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1582053079980_0032, Tracking URL = http://fbebe19f3edd:8088/proxy/application_1582053079980_0032/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1582053079980_0032
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-02-18 22:55:01,562 Stage-1 map = 0%,  reduce = 0%
2020-02-18 22:55:13,96

In [8]:
%%hive
INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM cont_ks;

INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM cont_ks;
Query ID = root_20200218225532_339a4060-a088-4fa8-88ca-cafe2ec5bfda
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1582053079980_0033, Tracking URL = http://fbebe19f3edd:8088/proxy/application_1582053079980_0033/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1582053079980_0033
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2020-02-18 22:55:51,408 Stage-1 map = 0%,  reduce = 0%
2020-02-18 22:56:02,479 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 3.63 sec
MapReduce Total cumulative CPU time: 3 seconds 630 msec
Ended Job = job_1582053079980_0033
Stage-3 is selected by condition resolver.
Stage-2 is filtered out by condition resolver.
Stage-4 is filtered out by condition resolver.
Moving data to directory hdfs://0.0.0.0:9000/tmp/output/.hive-staging_hive

In [9]:
!hive -S -e 'source output.hql'

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
File: output.hql is not a file.


In [10]:
!hadoop fs -copyToLocal /tmp/output output
!ls output/*

output/000000_0


In [11]:
!cat output/000000_0

aaa,13
bbb,16
ccc,23
ddd,23
eee,15
fff,20
ggg,13
hhh,16
iii,18
jjj,18
